# Standard Least Squared
- No preprosessing(not even polynomials), plotting or anything fancy
- Plain Least Squared from exercise 3

###### Next steps: 

Low-fruit:
- make polynomials
- implement cross validation

High fruit:
- Maybe make a new document with only the Exploratory data analysis with PCA and such. Then we different algorithms can choose to highlight some result and use it in their Feature prosessing. (https://en.wikipedia.org/wiki/Exploratory_data_analysis)
- find or write code that finds the best degrees() can't plot...or can just plot one dimension at a time, better to just use ridge regression at that point? Does LS have any type of overfitting add-on

### Libraries and imports

In [1]:
# standard libraries
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

# own functions
from proj1_helpers import * 
import basic_functions as bf
#import plot_functions as pf 

#constants
train_path = 'train.csv'
test_path = 'test.csv'

### Importing data

In [2]:
y, x, ids = load_csv_data(train_path, sub_sample=False) #remember to switch of subsample when running it "for real"

In [12]:

cleaned_x = bf.clean_data(x, replace_no_measure_with_mean=False, replace_no_measure_with_median=True )

cleaned_x

array([[  1.38470000e+02,   5.16550000e+01,   9.78270000e+01, ...,
          1.24000000e+00,  -2.47500000e+00,   1.13497000e+02],
       [  1.60937000e+02,   6.87680000e+01,   1.03235000e+02, ...,
         -1.00000000e-02,  -2.00000000e-03,   4.62260000e+01],
       [  1.12406000e+02,   1.62172000e+02,   1.25953000e+02, ...,
         -1.00000000e-02,  -2.00000000e-03,   4.42510000e+01],
       ..., 
       [  1.05457000e+02,   6.05260000e+01,   7.58390000e+01, ...,
         -9.99000000e+02,  -9.99000000e+02,   4.19920000e+01],
       [  9.49510000e+01,   1.93620000e+01,   6.88120000e+01, ...,
         -9.99000000e+02,  -9.99000000e+02,   0.00000000e+00],
       [ -9.99000000e+02,   7.27560000e+01,   7.08310000e+01, ...,
         -9.99000000e+02,  -9.99000000e+02,   0.00000000e+00]])

In [5]:
norm_x=bf.normalize(cleaned_x)


NameError: name 'cleaned_x' is not defined

### Split data

In [4]:
#seed = 1
#ratio = 0.8

#x_tr, x_te, y_tr, y_te = bf.split_data(x, y, ratio, seed)
#print(x_tr.shape, x_te.shape, y_tr.shape, y_te.shape)

### Exploratory data analysis (ex: PCA)
https://en.wikipedia.org/wiki/Exploratory_data_analysis

### Feature Processing (ex:Fourier, Poly)
http://machinelearningmastery.com/discover-feature-engineering-how-to- engineer-features-and-how-to-get-good-at-it/.

Ideas:
- standardize
- polynomials
- fourier transformation
- sine, cos, tan ...ect
- look up kvantemechanics formulas an see if we can find some correlations

### Train and produce weigths 

In [18]:
initial_w = np.zeros(len(x[1]))
max_iters = 100 # low
gamma = 0.1
tx = np.transpose(x) 
ty = np.transpose(y)

losses, w = bf.least_squares(ty, x)

print(w.shape)


(30,)


### Optimalizing: determining overfit or underfit
http://cs229.stanford.edu/materials/ML-advice.pdf

### Visualize, find pattern of wrong predictions -> back to Exploratory/Feature Proc
http://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf

### Cross validation MAGIC 


In [ ]:
#w_ridge, rmse_ridge=      bf.ridge_regression(y_tr, x_tr, lambda_)

#loss_reglogreg, w_reglogreg, losses_reglogreg=    bf.reg_logistic_regression(y_tr, tx_tr, lambda_, initial_w, max_iters, gamma)

#*args = hyperparameters 

def cross_validation(function_to_run, y, x, num_of_k_fold, *args):
    
    losses = []
    pred_acc_percents = []
    
    k_indices = build_k_indices(y, num_of_k_fold, 1)
    
    for k in range(num_of_k_fold):
        
        x_test, y_test, x_tr, y_tr = split_k(x,y,k_indices, k)
        
        if(function_to_run.__name__ == "reg_logistic_regression"):
            
            lambda_ = args[0]
            initial_w = args[1]
            max_iters = args[2]
            gamma = args[3]
            
            loss, weights, losses =  function_to_run(y_tr, x_tr, lambda_, initial_w, max_iters, gamma) 
            
        elif(function_to_run.__name__ == "ridge_regression"):
            
            lambda_ = args[0]
            
            weights, loss = bf.ridge_regression(y, x, lambda_)          
            
            
        losses.append(loss)

        pred_y = bf.log_pred(x, weights)    
        pred_acc_percent, soppel = bf.log_pred_acc(y, pred_y)
        pred_acc_percents.append(pred_acc_percent)
     
    
    loss_sum = 0
    for loss in losses:
        loss_sum += loss
    avg_loss = loss_sum / len(losses)
    
    acc_sum = 0
    for acc in pred_acc_percents:
        acc_sum += acc
    avg_acc = acc_sum / len(pred_acc_percents)
    
    return avg_loss, losses, avg_acc, pred_acc_percents 







#avg_loss, losses, avg_preds, pred_acc_percents = cross_validation(ridge_regression, y, x, k_folds, lambda_)

ready_x = norm_x # np.delete(norm_x, [5, 6, 12, 24,25,26,27,28,29], axis=1) 

print(ready_x.shape)

lambda_=0.5
initial_w = np.zeros((ready_x.shape[1], 1))

print(initial_w.shape)

max_iters = 20
gamma = 0.00001
k_folds = 3

avg_loss, losses, avg_preds, pred_acc_percents = cross_validation(bf.reg_logistic_regression, y, ready_x, k_folds, lambda_, initial_w, max_iters, gamma)

print("avg loss: ", avg_loss)
print("Average prediction NON-accuracy: ", avg_preds)


(250000, 30)
(30, 1)


In [10]:
def build_k_indices(y, k_fold, seed=1):
    """build k indices for k-fold."""
    num_row = y.shape[0]
    interval = int(num_row / k_fold)
    np.random.seed(seed)
    indices = np.random.permutation(num_row)
    k_indices = [indices[k * interval: (k + 1) * interval]
                 for k in range(k_fold)]
    return np.array(k_indices)

def split_k(x,y,k_indices, k):
    #Given the data x, y, k_indices build by build_k_idiices, and k
    #Splits the data into training and test data, where the k-th fold is the test data
    x_test, y_test= x[k_indices[k]], y[k_indices[k]]
    train_ind=np.delete(k_indices,k,0)
    train_ind=np.ravel(train_ind)
    x_tr, y_tr= x[train_ind], y[train_ind]
    return x_test, y_test, x_tr, y_tr 




### Predict higgs boson

In [ ]:
# Import test.csv which contains nonlabeled data
# Remember to have sub_sample=False when submitting to kaggle WARNING: it takes a looooong time
pred_y, pred_x, pred_ids = load_csv_data(test_path, sub_sample=False)
print(pred_x, pred_y)
print(pred_ids)

In [ ]:
weights = w

#predictions = compute_prediction(pred_x, weights)
#print(predictions[1:25])

y_predictions = predict_labels(weights, pred_x)
print(y_predictions[1:25])


In [3]:
# creates a file with the name you want in the folder of the Python-file. just look in the folder :) 
name = 'powerpuff_testing.csv'
create_csv_submission(pred_ids, y_predictions, name)

print("yolo")

#
# THE END
#

NameError: name 'pred_ids' is not defined